


# Assignment 5 on Natural Language Processing

## Date : 3rd Nov, 2020

### Instructor : Prof. Sudeshna Sarkar

### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Anusha Potnuru, Uppada Vishnu

The central idea of this assignment is to explore various language models specifically LSTM based and transformer. We will explore how the size of the model effects the sequence generated. We will see both character based and word based models.



Please submit with outputs. Submissions without predicted outputs will be penalized.

# Word Based LSTM model

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import numpy
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras

Do basic pre processing which includes lowering etc
Check the dataset and apply suitable preprocessing.

In [2]:
# Load the data and preprocess data and store corpus in raw_text
f=open('corpus.txt','rt',encoding="utf8")
raw_text=f.read()
f.close()
raw_text = raw_text.lower()
print(raw_text)

chapter i.
down the rabbit-hole alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, “and what is the use of a book,” thought alice “without pictures or conversations?” so she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a white rabbit with pink eyes ran close by her.
there was nothing so _very_ remarkable in that; nor did alice think it so _very_ much out of the way to hear the rabbit say to itself, “oh dear!
oh dear!
i shall be late!” (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural); but when the rabbit actually _took a watch out of its waist

In [3]:
import string
res = raw_text.translate(str.maketrans('', '', string.punctuation))
print(res)

chapter i
down the rabbithole alice was beginning to get very tired of sitting by her sister on the bank and of having nothing to do once or twice she had peeped into the book her sister was reading but it had no pictures or conversations in it “and what is the use of a book” thought alice “without pictures or conversations” so she was considering in her own mind as well as she could for the hot day made her feel very sleepy and stupid whether the pleasure of making a daisychain would be worth the trouble of getting up and picking the daisies when suddenly a white rabbit with pink eyes ran close by her
there was nothing so very remarkable in that nor did alice think it so very much out of the way to hear the rabbit say to itself “oh dear
oh dear
i shall be late” when she thought it over afterwards it occurred to her that she ought to have wondered at this but at the time it all seemed quite natural but when the rabbit actually took a watch out of its waistcoatpocket and looked at it an

In [4]:
# Hyperparameters of the model
vocab_size = 3000 # choose based on statistics
oov_tok = '<OOV>'
embedding_dim = 100
padding_type='post'
trunc_type='post'

In [5]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts([res])
word_index = tokenizer.word_index

In [6]:
seq_length = 50
tokens = tokenizer.texts_to_sequences([res])[0]

In [7]:
dataX = []
dataY = []

for i in range(0, len(tokens) - seq_length-1 , 1):
  seq_in = tokens[i:i + seq_length]
  seq_out = tokens[i + seq_length]

  if seq_out==1: #Skip samples where target word is OOV
    continue
    
  dataX.append(seq_in)
  dataY.append(seq_out)
 
N = len(dataX)
print ("Total training data size: ", N)

Total training data size:  25852


In [8]:
X = numpy.array(dataX)

# one hot encode the output variable
y = numpy.array(dataY)
y = np_utils.to_categorical(dataY)

In [9]:
# with embedding
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=seq_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(vocab_size, activation='softmax')
])
# compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 100)           300000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               84480     
_________________________________________________________________
dense (Dense)                (None, 3000)              387000    
Total params: 771,480
Trainable params: 771,480
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Use validation split of 0.2 while training
model.fit(X, y, epochs=20, batch_size=128, validation_split=0.2 ) 

Epoch 1/20
162/162 [==============================] - 3s 20ms/step - loss: 6.6472 - accuracy: 0.0535 - val_loss: 6.1813 - val_accuracy: 0.0895
Epoch 2/20
162/162 [==============================] - 2s 15ms/step - loss: 6.1734 - accuracy: 0.0552 - val_loss: 6.0906 - val_accuracy: 0.0895
Epoch 3/20
162/162 [==============================] - 2s 15ms/step - loss: 6.0555 - accuracy: 0.0581 - val_loss: 6.0733 - val_accuracy: 0.0932
Epoch 4/20
162/162 [==============================] - 2s 15ms/step - loss: 5.9539 - accuracy: 0.0626 - val_loss: 6.0502 - val_accuracy: 0.0940
Epoch 5/20
162/162 [==============================] - 2s 15ms/step - loss: 5.8606 - accuracy: 0.0676 - val_loss: 6.0410 - val_accuracy: 0.0849
Epoch 6/20
162/162 [==============================] - 2s 15ms/step - loss: 5.7670 - accuracy: 0.0708 - val_loss: 6.0028 - val_accuracy: 0.0963
Epoch 7/20
162/162 [==============================] - 2s 15ms/step - loss: 5.6585 - accuracy: 0.0801 - val_loss: 5.9569 - val_accuracy: 0.1021

In [11]:
reverse_word_map = dict((value, key) for (key, value) in word_index.items())
print(reverse_word_map)

{1: '<OOV>', 2: 'the', 3: 'and', 4: 'to', 5: 'a', 6: 'she', 7: 'of', 8: 'said', 9: 'it', 10: 'alice', 11: 'in', 12: 'was', 13: 'you', 14: 'i', 15: 'as', 16: 'her', 17: 'that', 18: 'at', 19: 'on', 20: 'had', 21: 'with', 22: 'all', 23: 'for', 24: 'very', 25: 'be', 26: 'so', 27: 'this', 28: 'little', 29: 'but', 30: '“i', 31: 'not', 32: 'they', 33: 'out', 34: 'he', 35: 'down', 36: 'his', 37: 'what', 38: 'about', 39: 'up', 40: 'one', 41: 'is', 42: 'went', 43: 'herself', 44: 'were', 45: 'like', 46: 'could', 47: 'would', 48: 'have', 49: 'thought', 50: 'when', 51: 'no', 52: 'if', 53: 'them', 54: 'or', 55: 'again', 56: 'into', 57: 'there', 58: 'then', 59: 'queen', 60: 'see', 61: 'off', 62: 'do', 63: 'time', 64: 'know', 65: 'your', 66: 'king', 67: 'its', 68: 'began', 69: 'by', 70: 'an', 71: 'my', 72: 'mock', 73: '“and', 74: 'did', 75: 'quite', 76: 'gryphon', 77: 'me', 78: 'who', 79: 'hatter', 80: 'turtle', 81: 'don’t', 82: 'their', 83: 'some', 84: 'way', 85: 'just', 86: 'how', 87: 'more', 88: 'i

In [12]:
# Complete the code to return next n words greedily
def next_tokens(input_str, n): 
	print ("Seed: \n",  input_str)
	final_string=input_str
	pattern= tokenizer.texts_to_sequences([input_str])[0]
	for i in range(n):
		x=numpy.reshape(pattern,(1,len(pattern)))
		prediction = model.predict(x, verbose=0)
		p=numpy.argmax(prediction)
		# get next word index. Use reverse_word_map to get the word
		word=reverse_word_map[p]
		final_string= final_string+" "+word
		pattern.append(p)
		pattern=pattern[1:]
	return final_string

In [13]:
# pick a random seed

start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ' '.join([reverse_word_map[value] for value in pattern])

print(next_tokens( input_str , 10))

Seed: 
 her foot she was a good deal frightened by this very sudden change but she felt that there was no time to be lost as she was shrinking rapidly so she set to work at once to eat some of the other bit her chin was pressed so closely against
her foot she was a good deal frightened by this very sudden change but she felt that there was no time to be lost as she was shrinking rapidly so she set to work at once to eat some of the other bit her chin was pressed so closely against the queen of the cat and the caterpillar was a


In [14]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

# Use first 50 tokens from given input_str as input.(Use tokenizer to split to take first 50)
print(next_tokens( input_str , 10))

Seed: 
 The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not  a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue  him or his sheep.
The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not  a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue  him or his sheep. and the queen was a little little little little little


# Character based LSTM Model 1

In [15]:
punc = '''!()-[]{};:\<>?@#$%^&*_~'''
for ele in raw_text:  
    if ele in punc:  
        raw_text = raw_text.replace(ele, "")  
print(raw_text)

chapter i.
down the rabbithole alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, “and what is the use of a book,” thought alice “without pictures or conversations” so she was considering in her own mind as well as she could, for the hot day made her feel very sleepy and stupid, whether the pleasure of making a daisychain would be worth the trouble of getting up and picking the daisies, when suddenly a white rabbit with pink eyes ran close by her.
there was nothing so very remarkable in that nor did alice think it so very much out of the way to hear the rabbit say to itself, “oh dear
oh dear
i shall be late” when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural but when the rabbit actually took a watch out of its waistcoatpocket, and lo

In [16]:
# User the preprocess data and create raw_text

# create mapping of unique characters to integers
chars = sorted(list(set(raw_text)))

char_to_int = dict((c, i) for i, c in enumerate(chars))

In [17]:
# Print the total characters and character vacob size
n_chars = len(raw_text)
n_vocab = len(chars)

In [18]:
'''
Prepare dataset where the input is sequence of 100 characters and target is next character.
'''
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    # Write code here
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  140599


In [19]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

# one hot encode the output variable
dataY = numpy.array(dataY)
y = np_utils.to_categorical(dataY)

In [20]:
embedding_dim =100
max_length =100

In [21]:
from keras.layers import Embedding
model = Sequential()
model.add(Embedding(n_vocab, embedding_dim, input_length=max_length))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          3600      
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 36)                9252      
Total params: 378,420
Trainable params: 378,420
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(X, y, epochs=20, batch_size=128)

Epoch 1/20
1099/1099 [==============================] - 20s 18ms/step - loss: 2.1678
Epoch 2/20
1099/1099 [==============================] - 20s 18ms/step - loss: 1.6817
Epoch 3/20
1099/1099 [==============================] - 20s 18ms/step - loss: 1.4926
Epoch 4/20
1099/1099 [==============================] - 20s 18ms/step - loss: 1.3790
Epoch 5/20
1099/1099 [==============================] - 20s 18ms/step - loss: 1.2969
Epoch 6/20
1099/1099 [==============================] - 20s 18ms/step - loss: 1.2375
Epoch 7/20
1099/1099 [==============================] - 20s 18ms/step - loss: 1.1874
Epoch 8/20
1099/1099 [==============================] - 20s 18ms/step - loss: 1.1459
Epoch 9/20
1099/1099 [==============================] - 20s 18ms/step - loss: 1.1057
Epoch 10/20
1099/1099 [==============================] - 20s 18ms/step - loss: 1.0708
Epoch 11/20
1099/1099 [==============================] - 20s 18ms/step - loss: 1.0363
Epoch 12/20
1099/1099 [==============================] - 20s 18

In [23]:
#implement mapping of integer to character
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [24]:
'''
Complete code below to get the generated string using the model.
'''
def predict_next_100_chars(pattern,x):
	final_string=''
	seq=pattern[0:100]
	data=[]
	data=[char_to_int[char] for char in seq]
	for i in range(x):
		# Complete Code
		l = numpy.reshape(data, (1, len(data), 1))
		prediction = model.predict(l, verbose=0)
		index = numpy.argmax(prediction)
		data.append(index)
		data = data[1:len(data)]
		final_string=final_string+int_to_char[index]

	return final_string

In [25]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])

print(predict_next_100_chars(input_str,200))

t the caterpillar continued to herself “the dormouse seems tone,” said the king “i think i can tell you my history, and the mock turtle say you could not go and beginning to see the mock turtle say yo


In [26]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

# Use first 100 characeters from given input_str as input and generate next 200 characters.
input_str=input_str.lower()

print(predict_next_100_chars(input_str,200))

nd she had not gone the queen of the court, and the mock turtle said “the dormouse seems tone,” said the king “i think i can tell you my history, and the mock turtle say you could not go and beginning


## Character based LSTM Model 2


In [27]:
model1 = Sequential()
model1.add(Embedding(n_vocab, embedding_dim, input_length=max_length))
model1.add(LSTM(256, input_shape=(X.shape[1], embedding_dim),return_sequences=True))
model1.add(Dropout(0.2))
model1.add(LSTM(256))
model1.add(Dropout(0.2))
model1.add(Dense(y.shape[1], activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam')

In [28]:
model1.fit(X, y, epochs=20, batch_size=64)

Epoch 1/20
2197/2197 [==============================] - 50s 23ms/step - loss: 2.0670
Epoch 2/20
2197/2197 [==============================] - 49s 22ms/step - loss: 1.5269
Epoch 3/20
2197/2197 [==============================] - 50s 23ms/step - loss: 1.3635
Epoch 4/20
2197/2197 [==============================] - 50s 23ms/step - loss: 1.2666
Epoch 5/20
2197/2197 [==============================] - 50s 23ms/step - loss: 1.1978
Epoch 6/20
2197/2197 [==============================] - 50s 23ms/step - loss: 1.1509
Epoch 7/20
2197/2197 [==============================] - 50s 23ms/step - loss: 1.1071
Epoch 8/20
2197/2197 [==============================] - 50s 23ms/step - loss: 1.0671
Epoch 9/20
2197/2197 [==============================] - 49s 22ms/step - loss: 1.0342
Epoch 10/20
2197/2197 [==============================] - 49s 22ms/step - loss: 1.0071
Epoch 11/20
2197/2197 [==============================] - 49s 22ms/step - loss: 0.9849
Epoch 12/20
2197/2197 [==============================] - 49s 22

In [29]:
# Generate the sequence similar to above methods

'''
Complete code below to get the generated string using the model.
'''
def predict_next_100_chars(pattern,x):
	final_string=''
	seq=pattern[0:100]
	data=[]
	data=[char_to_int[char] for char in seq]
	for i in range(x):
		# Complete Code
		l = numpy.reshape(data, (1, len(data), 1))
		prediction = model1.predict(l, verbose=0)
		index = numpy.argmax(prediction)
		data.append(index)
		data = data[1:len(data)]
		final_string=final_string+int_to_char[index]

	return final_string

In [30]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])

print(predict_next_100_chars(input_str,200))

hing to begin with the sea.
the mock turtle was a little sister was the story “it’s a little sisters,” said the mock turtle.
“then you shouldn’t be so she set to be a parts of things it was a little s


In [31]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

 # Use first 100 characeters from given input_str as input and generate next 200 characters.
input_str=input_str.lower()
 
print(predict_next_100_chars(input_str,200))

nd she said this she had nothing off to see it was a little sister, and then she was so much as she could not think that she was so much as she could not think that she was so much as she could not th


# Questions

**Question:** What are your observations based on the model(all) outputs on train data(in domain) vs unseen data(out of domain) ?

**Answer:**
When we use the model on trained data the prediction is more relevant with the context. Since we trained on a corpus with little data there are many contexts which are not encountered by our model. As a result the prediction on Unseen Data is not contextually relevant.

**Question:** What did you observe in the outputs of char LSTM model1 vs char LSTM model2 ?

**Answer:**
We observe that the outputs of char LSTM model 2 is better. This is because it consists of stacked LSTM as a result the encoding is better.

# Bonus (Not to be graded)

## Transformer based language model (Bert)
You can run the below cells to predict the next word using pretrained BERT model. 

In [32]:
!pip install transformers

     |████████████████████████████████| 1.3MB 13.2MB/s 
     |████████████████████████████████| 2.9MB 53.7MB/s 
     |████████████████████████████████| 1.1MB 57.3MB/s 
     |████████████████████████████████| 890kB 54.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=33ffc5faa33a5cdc7ef21e9995a770727d2995897e5c37066ee599e5c32e612c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [33]:
import os
import torch
import string
from transformers import BertTokenizer, BertForMaskedLM

In [34]:
def load_model(model_name):
  try:
    if model_name.lower() == "bert":
      bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
      bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased').eval()
      return bert_tokenizer,bert_model
  except Exception as e:
    pass

In [35]:
def decode(tokenizer, pred_idx, top_clean):
  ignore_tokens = string.punctuation + '[PAD]'
  tokens = []
  for w in pred_idx:
    token = ''.join(tokenizer.decode(w).split())
    if token not in ignore_tokens:
      tokens.append(token.replace('##', ''))
  return '\n'.join(tokens[:top_clean])

In [36]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
  text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
  # if <mask> is the last token, append a "." so that models dont predict punctuation.
  if tokenizer.mask_token == text_sentence.split()[-1]:
    text_sentence += ' .'
  input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
  mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
  return input_ids, mask_idx

In [37]:
def get_all_predictions(text_sentence, top_clean=5):
  input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
  with torch.no_grad():
    predict = bert_model(input_ids)[0]
    print(predict.shape)
    
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)
  return {'bert': bert}

In [38]:
def get_prediction_eos(input_text):
  try:
    input_text += ' <mask>'
    res = get_all_predictions(input_text, top_clean=int(top_k))
    return res
  except Exception as error:
    pass

In [39]:
# Below code predicts the next top_k words. You can modify this to get next n words using top_k=1 and greedy decoding it. 
top_k= 3
print('Predict next ', top_k, ' words')
model_name = 'BERT'
bert_tokenizer, bert_model  = load_model(model_name) 
input_text = "" ### GIVE YOUR INPUT STRING HERE
res = get_prediction_eos(input_text)
answer = []
print(res['bert'].split("\n"))
for i in res['bert'].split("\n"):
  answer.append(i)
  answer_as_string = "    ".join(answer)

print(answer_as_string)

Predict next  3  words


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([1, 4, 30522])
['p', 'vol', 'math']
p    vol    math
